In [1]:
import torch
import numpy as np
import time
import os
from utils import HDDataset
from model import BModel
from torchvision import datasets, transforms
import scipy
from encoder import RandomFourierEncoder
import main2
import importlib
importlib.reload(main2)
import csv
from scipy.io import savemat, loadmat
from torch.utils.data import DataLoader, Subset
import random

In [2]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
gorder = 2
dim = 10000
gamma = 0.3
lr = 0.01
classes = 10
channels = 3
epochs = 3
train = main2.train
test = main2.test

In [5]:
trainset = datasets.CIFAR10(root='../../Data', train=True, download=True, transform=transforms.ToTensor())
testset = datasets.CIFAR10(root='../../Data', train=False, download=True, transform=transforms.ToTensor())
n_splits = 20
split_size = len(testset) // n_splits
accuracies1 = np.zeros((11, n_splits))
hyperdims = loadmat('../EHDGNet_CIFAR10_nHD.mat')['EHDGNet_CIFAR10_nHD']
hyperdims = np.mean(hyperdims, axis=1, dtype=int) // 3
print(hyperdims*3)
input_dim = torch.prod(torch.tensor(list(trainset[0][0].size())))
print(input_dim)

Files already downloaded and verified
Files already downloaded and verified
[15000 17250 18999 21249 23250 25248 27000 28998 30999 33000 34998]
tensor(3072)


In [6]:
for i, nHD in enumerate(hyperdims):
    print(f'nHD: {nHD*3} ------------')
    encoder = RandomFourierEncoder(input_dim, gamma, gorder, output_dim=nHD)
    encoder.build_item_mem()
    train_hd, y_train = encoder.encode_data_extract_labels(trainset)
    model = BModel(in_dim=nHD, classes=10)
    train_dataset = HDDataset(train_hd, y_train)
    trainloader = DataLoader(train_dataset, batch_size=1024, shuffle=False, pin_memory=True, num_workers=2)
    for split_idx in range(n_splits):
        # Test
        start_idx = split_idx * split_size
        end_idx = start_idx + split_size
        split_subset = Subset(testset, range(start_idx, end_idx))
        test_hd, y_test = encoder.encode_data_extract_labels(split_subset)
        if split_idx == 0:
            print(test_hd.shape)
        test_dataset = HDDataset(test_hd, y_test)
        testloader = DataLoader(test_dataset, batch_size=1024, shuffle=False, pin_memory=True, num_workers=2)
        accuracies1[i, split_idx] = train(trainloader, testloader, lr, nHD)
        print(accuracies1[i, split_idx], 'Split Index: ', split_idx, 'Accuracy')
        del testloader
        del test_dataset
        del test_hd
        
    del train_hd
    del trainloader
    del train_dataset
    print(f'nHD: {nHD*3} ---------------------------------------------- Finished')
    print(f'Average Accuracy for nHD={nHD*3}: {np.mean(accuracies1[i, :])}')

nHD: 15000 ------------
the threshold to discretize fourier features to group elements tensor([0.])
torch.Size([500, 15000])
18.2 Split Index:  0 Accuracy
18.4 Split Index:  1 Accuracy
17.2 Split Index:  2 Accuracy
18.0 Split Index:  3 Accuracy
19.0 Split Index:  4 Accuracy
17.4 Split Index:  5 Accuracy
17.8 Split Index:  6 Accuracy
15.0 Split Index:  7 Accuracy
19.4 Split Index:  8 Accuracy
16.4 Split Index:  9 Accuracy
15.2 Split Index:  10 Accuracy
16.0 Split Index:  11 Accuracy
15.0 Split Index:  12 Accuracy
16.0 Split Index:  13 Accuracy
19.4 Split Index:  14 Accuracy
18.2 Split Index:  15 Accuracy
17.6 Split Index:  16 Accuracy
18.0 Split Index:  17 Accuracy
19.6 Split Index:  18 Accuracy
16.2 Split Index:  19 Accuracy
nHD: 15000 ---------------------------------------------- Finished
Average Accuracy for nHD=15000: 17.400000000000002
nHD: 17250 ------------
the threshold to discretize fourier features to group elements tensor([0.])
torch.Size([500, 17250])
24.2 Split Index:  0 A

In [7]:
np.mean(accuracies1, axis=1)

array([17.4 , 24.54, 24.62, 22.01, 25.56, 23.47, 23.14, 27.3 , 23.35,
       26.56, 26.18])

In [8]:
savemat('RFFHDC_CIFAR10.mat', {'RFFHDC_CIFAR10': accuracies1})